In [31]:
import pandas as pd
import numpy as np

In [32]:
data = pd.read_csv('flats_berlin.csv')

## Data Check

### Check for duplicated rows

In [33]:
data[data.duplicated()]

,Titel,Url,Kaltmiete,Nebenkosten,Heizkosten,Gesamtmiete,Kaution o. Genossenschaftsanteile,Etage,Wohnfläche ca.,Zimmer,...,Baujahr,Modernisierung/ Sanierung,Objektzustand,Heizungsart,Wesentliche Energieträger,Energieausweis,Energieausweistyp,Energieverbrauchskennwert,Anbieter,Adreese


### Check percentage of missing values

In [34]:
data.isna().mean()

Titel                                0.000000
Url                                  0.000000
Kaltmiete                            0.000000
Nebenkosten                          0.000000
Heizkosten                           0.000000
Gesamtmiete                          0.000000
Kaution o. Genossenschaftsanteile    0.073649
Etage                                0.105405
Wohnfläche ca.                       0.000000
Zimmer                               0.000000
Schlafzimmer                         0.427365
Badezimmer                           0.271959
Personenaufzug                       0.476014
Einbauküche                          0.359797
Baujahr                              0.121959
Modernisierung/ Sanierung            0.698649
Objektzustand                        0.226014
Heizungsart                          0.151351
Wesentliche Energieträger            0.232770
Energieausweis                       0.284797
Energieausweistyp                    0.314189
Energieverbrauchskennwert         

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2960 entries, 0 to 2959
Data columns (total 24 columns):
Titel                                2960 non-null object
Url                                  2960 non-null object
Kaltmiete                            2960 non-null object
Nebenkosten                          2960 non-null object
Heizkosten                           2960 non-null object
Gesamtmiete                          2960 non-null object
Kaution o. Genossenschaftsanteile    2742 non-null object
Etage                                2648 non-null object
Wohnfläche ca.                       2960 non-null object
Zimmer                               2960 non-null object
Schlafzimmer                         1695 non-null float64
Badezimmer                           2155 non-null float64
Personenaufzug                       1551 non-null object
Einbauküche                          1895 non-null object
Baujahr                              2599 non-null object
Modernisierung/ Sanie

## Data Transformation

Showcasing of different methods how data can be manipulated:

- str.replace
- .replace with regex
- loc selection and assignment
- custom functions
- lambda functions

In [36]:
data['Kaltmiete'] = pd.to_numeric(data['Kaltmiete'].str.replace('.','').str.replace(',','.').str.replace(' €',''))

In [37]:
data['Nebenkosten'] = data['Nebenkosten'].str.replace('.','').str.replace(',','.').str.replace(' €','')
data.loc[data['Nebenkosten'] == 'keine Angabe', 'Nebenkosten' ] = np.nan
data['Nebenkosten'] = pd.to_numeric(data['Nebenkosten'])

In [38]:
def transform_heizkosten(x):
    x = x.replace('.','').replace(',','.').replace(' €','')
    if x == 'in Nebenkosten enthalten' or 'inkl' in x:
        x = 0
    elif x == 'nicht in Nebenkosten enthalten' or x == 'keine Angabe':
        x = np.nan
    else:
        x
    return x

data['Heizkosten'] = pd.to_numeric(data['Heizkosten'].apply(transform_heizkosten))

In [39]:
data['Gesamtmiete'] = data['Gesamtmiete'].replace({r" €":"", r"\.":""},regex=True).replace({r"\,":"."},regex=True)
data['Gesamtmiete'] = pd.to_numeric(data['Gesamtmiete'].replace(to_replace=[r" \(zzgl Nebenkosten & Heizkosten\)",
                                                     r" \(zzgl Heizkosten\)",
                                                     r" \(zzgl Nebenkosten\)"],value='',regex=True))

In [40]:
data['Wohnfläche ca.'] = pd.to_numeric(
    data['Wohnfläche ca.'].apply(lambda string: string.replace(' m²','').replace('.','').replace(',','.')))

In [41]:
data['Kaution o. Genossenschaftsanteile'] = data['Kaution o. Genossenschaftsanteile'].replace(to_replace= [r" EUR",r" \€", r"\.",r"\€"],
                                                  value= '',
                                                  regex=True).replace().replace('\,','.',regex=True)

In [42]:
# To be further transformed if necessary
data['Kaution o. Genossenschaftsanteile'].sort_values(ascending=False).head(100)

2747            zwei Nettokaltmieten
1792                            nein
1323                  nach Absprache
2514                        entfällt
765             drei Nettokaltmieten
                    ...             
2712    Kaution: 3 Netto-Kalt-Mieten
1384    Kaution: 3 Netto-Kalt-Mieten
2653    Kaution: 3 Netto-Kalt-Mieten
2652    Kaution: 3 Netto-Kalt-Mieten
1491    Kaution: 3 Netto-Kalt-Mieten
Name: Kaution o. Genossenschaftsanteile, Length: 100, dtype: object

In [43]:
data['Baujahr'] = pd.to_numeric(data['Baujahr'].replace("unbekannt",''))

In [44]:
data['Modernisierung/ Sanierung'] = pd.to_numeric(data['Modernisierung/ Sanierung'].replace(r"zuletzt ",'',regex=True))

In [45]:
data["Energieverbrauchskennwert"] = pd.to_numeric(data["Energieverbrauchskennwert"].replace(r" kWh/\(m\²\*a\)","",regex=True).replace(r"\.","",regex=True).replace(r"\,",".",regex=True))

In [46]:
# Correcting small spelling mistake
data.rename(columns={'Adreese':'Adresse'},inplace=True)

In [47]:
def split_address(x):
    address_list = x.split(',')
    length = len(address_list)
    street = np.nan
    zip_code = np.nan
    kiez = np.nan
    if length == 3:
        street = address_list[0].strip()
        zip_code = address_list[1].strip()
        kiez = address_list[2].strip()
        #return street, zip_code, kiez
    if length == 2:
        street = np.nan
        zip_code = address_list[0].strip()
        kiez = address_list[1].strip()
        kiez = kiez.replace(' Die vollständige Adresse der Immobilie erhalten Sie vom Anbieter.','').strip()
    
    return street, zip_code, kiez

address = pd.DataFrame.from_records(data['Adresse'].apply(split_address)).rename(columns={0:'Street',1:'PLZ',2:'Kiez'})
data = data.join(address)

## Feature Engineering

In [48]:
data['Kaltmiete pro m²'] = data['Kaltmiete']/data['Wohnfläche ca.']

In [49]:
"""
Erstmalige Bezugsfertigkeit der Wohnung und Ausstattung	Mietpreis pro Quadratmeter
bis 1918 mit Sammelheizung und mit Bad	6,45 Euro
bis 1918 mit Sammelheizung oder mit Bad	5,00 Euro
bis 1918 ohne Sammelheizung und ohne Bad	3,92 Euro
1919 bis 1949 mit Sammelheizung und mit Bad	6,27 Euro
1919 bis 1949 mit Sammelheizung oder mit Bad	5,22 Euro
1919 bis 1949 ohne Sammelheizung und ohne Bad	4,59 Euro
1950 bis 1964 mit Sammelheizung und mit Bad	6,08 Euro
1950 bis 1964 mit Sammelheizung oder mit Bad	5,62 Euro
1965 bis 1972 mit Sammelheizung und mit Bad	5,95 Euro
1973 bis 1990 mit Sammelheizung und mit Bad	6,04 Euro
1991 bis 2002 mit Sammelheizung und mit Bad	8,13 Euro
2003 bis 2013 mit Sammelheizung und mit Bad	9,80 Euro
"""

'\nErstmalige Bezugsfertigkeit der Wohnung und Ausstattung\tMietpreis pro Quadratmeter\nbis 1918 mit Sammelheizung und mit Bad\t6,45 Euro\nbis 1918 mit Sammelheizung oder mit Bad\t5,00 Euro\nbis 1918 ohne Sammelheizung und ohne Bad\t3,92 Euro\n1919 bis 1949 mit Sammelheizung und mit Bad\t6,27 Euro\n1919 bis 1949 mit Sammelheizung oder mit Bad\t5,22 Euro\n1919 bis 1949 ohne Sammelheizung und ohne Bad\t4,59 Euro\n1950 bis 1964 mit Sammelheizung und mit Bad\t6,08 Euro\n1950 bis 1964 mit Sammelheizung oder mit Bad\t5,62 Euro\n1965 bis 1972 mit Sammelheizung und mit Bad\t5,95 Euro\n1973 bis 1990 mit Sammelheizung und mit Bad\t6,04 Euro\n1991 bis 2002 mit Sammelheizung und mit Bad\t8,13 Euro\n2003 bis 2013 mit Sammelheizung und mit Bad\t9,80 Euro\n'

In [50]:
rental_ceiling_info = {
    'min_year': [1918, 1918, 1918, 1919, 1919, 1919, 1950, 1950, 1965, 1973, 1991, 2003 ],
    'max_year': [1918, 1918, 1918, 1949, 1949, 1949, 1964, 1964, 1972, 1990, 2002, 2013 ],
    'sammelheizung' : [True, True, False, True, True, False, True, True, True, True, True, True],
    'bad' : [True, True, False, True, True, False, True, True, True, True, True, True],
    'condition': ['and', 'or', 'and', 'and', 'or', 'and', 'and', 'or', 'and', 'and', 'and', 'and'],
    'price_per_square_meter' : [6.45, 5.00, 3.92, 6.27, 5.22, 4.59, 6.08, 5.62, 5.95, 6.04, 8.13, 9.8]
}

In [51]:
len(rental_ceiling_info['price_per_square_meter'])

12

In [52]:
rental_ceiling_info = pd.DataFrame.from_dict(rental_ceiling_info)

In [53]:
rental_ceiling_info

,min_year,max_year,sammelheizung,bad,condition,price_per_square_meter
0,1918,1918,True,True,and,6.45
1,1918,1918,True,True,or,5.00
2,1918,1918,False,False,and,3.92
3,1919,1949,True,True,and,6.27
4,1919,1949,True,True,or,5.22
5,1919,1949,False,False,and,4.59
6,1950,1964,True,True,and,6.08
7,1950,1964,True,True,or,5.62
8,1965,1972,True,True,and,5.95
9,1973,1990,True,True,and,6.04


In [54]:
def find_price_ceiling(df):
    if df['Baujahr'] <= 1918:
        max_price = 6.45
    elif df['Baujahr'] >= 1919 and df['Baujahr'] <= 1949:
        max_price = 6.27
    elif df['Baujahr'] >= 1950 and df['Baujahr'] <= 1964:
        max_price = 6.08
    elif df['Baujahr'] >= 1965 and df['Baujahr'] <= 1972:
        max_price = 5.95
    elif df['Baujahr'] >= 1973 and df['Baujahr'] <= 1990:
        max_price = 6.04
    elif df['Baujahr'] >= 1991 and df['Baujahr'] <= 2002:
        max_price = 8.13
    elif df['Baujahr'] >= 2003 and df['Baujahr'] <= 2013:
        max_price = 9.80
    else:
        max_price = np.nan
    
    return max_price

In [55]:
data['Mietendeckel pro Quadratmeter ohne Modernisierung']=  data.apply(find_price_ceiling,axis=1) * 1.2

In [56]:
illegal = data[data['Kaltmiete pro m²'] > data['Mietendeckel pro Quadratmeter ohne Modernisierung']]

In [ ]:
illegal.groupby('Anbieter').count().sort_values(by='Titel',ascending=False)

In [ ]:
not_illegal = data[data['Kaltmiete pro m²'] < data['Mietendeckel pro Quadratmeter ohne Modernisierung']]

In [ ]:
not_illegal.groupby('Anbieter').count().sort_values(by='Titel',ascending=False)